In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Llama-2-7b Setup

In [ ]:
!pip install llama-index deeplake
!pip install pypdf
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install install sentence_transformers
!pip install llama_index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.8/585.8 kB 55.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/

In [ ]:
import logging
import sys
from llama_index import (
    VectorStoreIndex,
    SimpleKeywordTableIndex,
    SimpleDirectoryReader,
    ServiceContext,
    download_loader,
    Document,
)
from llama_index.vector_stores import DeepLakeVectorStore
from llama_index.llms import OpenAI
from typing import List, Optional, Tuple
from pathlib import Path
import requests
import tqdm
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
import os
from google.colab import files

directory_name = "data"
os.makedirs(directory_name, exist_ok=True)

uploaded = files.upload()
for filename in uploaded.keys():
    os.rename(filename, os.path.join(directory_name, filename))


Saving Predicting Startup Survival from Digital Traces_ Towards a Proced (1).pdf to Predicting Startup Survival from Digital Traces_ Towards a Proced (1).pdf
Saving Resume_Murat_Ali_Avcu_NEW.pdf to Resume_Murat_Ali_Avcu_NEW.pdf


In [ ]:
folder_path = directory_name
file_names = os.listdir(folder_path)
file_paths = [os.path.join(folder_path, file_name) for file_name in file_names]


In [ ]:
integers = list(range(0, len(file_paths)))
print(integers)
zipped_data = list(zip(file_paths,integers))
PDFReader = download_loader("PDFReader")
loader = PDFReader()

[0, 1]


In [ ]:
import os
def download_reports(
    data: List[Tuple[str, int, int]], out_dir: Optional[str] = None
) -> List[Document]:
    """Download pages from a list of urls."""
    docs = []
    out_dir = Path(out_dir or ".")
    if not out_dir.exists():
        print(out_dir)
        os.makedirs(out_dir)

    for file_paths,integers in tqdm.tqdm(data):
        path_base = file_paths.split("/")[-1]
        out_path = out_dir / path_base
        if not out_path.exists():
            r = requests.get(file_paths)
            with open(out_path, "wb") as f:
                f.write(r.content)
        doc = loader.load_data(file=Path(out_path))[0]

        date_str =   "-01-" + str(integers)
        doc.extra_info = {"Date": date_str}

        docs.append(doc)
    return docs

docs = download_reports(zipped_data, "data")

100%|██████████| 2/2 [00:01<00:00,  1.46it/s]


In [ ]:
import torch
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts.prompts import SimpleInputPrompt
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

llm_llama = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage !!!!!!!!!!!!!!!!!!!!!!!! USE GPU !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import ServiceContext
from llama_index.embeddings import LangchainEmbedding
embed_model=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))
service_context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm_llama,
    embed_model=embed_model
)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from llama_index.storage.storage_context import StorageContext

dataset_root = "data2/data2"
vector_indices = {}
for idx, (_, year) in enumerate(zipped_data):
    doc = docs[idx]

    dataset_path = dataset_root + f"_{year}"
    vector_store = DeepLakeVectorStore(
        dataset_path=dataset_path,
        overwrite=True,
        verbose=False,
    )
    storage_context = StorageContext.from_defaults(vector_store=vector_store)

    vector_index = VectorStoreIndex.from_documents(
        [doc], storage_context=storage_context, service_context=service_context
    )
    vector_indices[(year)] = vector_index

Uploading data to deeplake dataset.


100%|██████████| 1/1 [00:00<00:00, 87.00it/s]


Uploading data to deeplake dataset.


100%|██████████| 1/1 [00:00<00:00, 226.67it/s]


In [ ]:
vector_indices

{0: <llama_index.indices.vector_store.base.VectorStoreIndex at 0x78ea50badb40>,
 1: <llama_index.indices.vector_store.base.VectorStoreIndex at 0x78ea50d525c0>}

In [ ]:
response = (
    vector_indices[(2)]
    .as_query_engine(service_context=service_context)
    .query("What is the most crucial point for start-up to survive")
)


In [ ]:
print(response)
#print(str(response))
print(response.get_formatted_sources())

Based on the information provided in the context, the most crucial point for a startup to survive is the feasibility of its business model. The study suggests that up to 78% of portfolio investments fail to survive for five years in the marketplace, highlighting the importance of assessing the feasibility of a startup's business model before investing. The authors of the study used machine learning algorithms to predict 5-year survival rates for startups based on digital traces, such as social media activity, and found that the feasibility of the business model is the most important factor in predicting survival. Therefore, it is crucial for startups to thoroughly evaluate their business model and ensure that it is feasible and viable in the marketplace before seeking investment from early stage investors.
Based on the information provided in the context, the most crucial point for a startup to survive is the feasibility of its business model. The study suggests that up to 78% of portf

# Calling Functions

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install scipy
!pip install tenacity
!pip install tiktoken
!pip install openai

In [ ]:
import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-3.5-turbo-1106"
client = OpenAI(api_key="sk-xxxx") # --------------------- YOUR API KEY ---------------------

In [ ]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    }
                },
                "required": ["location", "format", "num_days"]
            },
        }
    },


    {
        "type": "function",
        "function": {
            "name": "get_llama_response",
            "description": "get llama response",
            "parameters": {
                "type": "object",
                "properties": {
                    "user_input": {
                        "type": "string",
                        "description": "Query about the documents",
                    },
                    "index_number":{
                        "type":"integer",
                        "description": "index number of the document",
                    },

                },
                "required": ["user_input","index_number"]
            },
        }
    },



]

In [ ]:
from IPython.display import Markdown, display
def get_llama_response(user_input,index_number):
    response = (
    vector_indices[(int(index_number))]
    .as_query_engine(service_context=service_context)
    .query(f"{user_input}")
  )
    display(Markdown(f"<b>{str(response)}<b>"))
    strresponse=str(response)
    return strresponse

def get_current_weather (city,format):
  """
  --------Use API to get data--------

  """


  display(Markdown(f"<b>16°C°F Precipitation: 0%Humidity: 59%Wind: 11 km/hWeatherMonday 15:00Cloudy<b>"))


In [ ]:
llama_response_arguments = []
current_weather_arguments = []
while True:
  print("-"*100)
  usersss=input("")
  if usersss=="exit":
    break
  else:
    messages = []
    print("-"*100)
    #messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
    messages.append({"role": "user", "content": f"{usersss}"})
    chat_response = chat_completion_request(
        messages, tools=tools, model='gpt-3.5-turbo-1106'
    )

    assistant_message = chat_response.choices[0].message.tool_calls
    print(assistant_message)

    for tool_call in assistant_message:
        if tool_call.function.name == 'get_llama_response':
            print("Function get_llama_response is running...")
            llama_response_arguments.append(tool_call.function.arguments)
            for item in llama_response_arguments:
                parsed_arguments = json.loads(item)
                user_input = parsed_arguments.get('user_input')
                index_number = parsed_arguments.get('index_number')
                print(user_input,index_number)
                if user_input is not None:
                    user_inputs=user_input
                if index_number is not None:
                    index_number=index_number
                get_llama_response(user_inputs,index_number)
                #print(user_input,index_number)
        elif tool_call.function.name == 'get_current_weather':
            print("Fuunction get_current_weather is running...")
            current_weather_arguments.append(tool_call.function.arguments)
            for item in current_weather_arguments:
                parsed_arguments = json.loads(item)
                location = parsed_arguments.get('location')
                format = parsed_arguments.get('format')
                if location is not None:
                    location=location
                if format is not None:
                    format=format
                print(location,format)
                get_current_weather(location,format)


# Run

In [ ]:
from IPython.display import Markdown, display
llama_response_arguments = []
current_weather_arguments = []
while True:
  print("-"*100)
  usersss=input("")
  if usersss=="exit":
    break
  else:
    messages = []
    print("-"*100)
    #messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
    messages.append({"role": "user", "content": f"{usersss}"})
    chat_response = chat_completion_request(
        messages, tools=tools, model='gpt-3.5-turbo-1106'
    )

    assistant_message = chat_response.choices[0].message.tool_calls
    print(assistant_message)

    for tool_call in assistant_message:
        if tool_call.function.name == 'get_llama_response':
            print("Function get_llama_response is running...")
            llama_response_arguments.append(tool_call.function.arguments)
            for item in llama_response_arguments:
                parsed_arguments = json.loads(item)
                user_input = parsed_arguments.get('user_input')
                index_number = parsed_arguments.get('index_number')
                if user_input is not None:
                    user_inputs=user_input
                if index_number is not None:
                    index_number=index_number
                get_llama_response(user_inputs,index_number)
                #print(user_input,index_number)
        elif tool_call.function.name == 'get_current_weather':
            print("Function get_current_weather is running...")
            current_weather_arguments.append(tool_call.function.arguments)
            for item in current_weather_arguments:
                parsed_arguments = json.loads(item)
                location = parsed_arguments.get('location')
                format = parsed_arguments.get('format')
                if location is not None:
                    location=location
                if format is not None:
                    format=format
                #print(location,format)
                get_current_weather(location,format)


----------------------------------------------------------------------------------------------------
whats the weather in Istanbul
----------------------------------------------------------------------------------------------------
[ChatCompletionMessageToolCall(id='call_GWugXh2EC9gRhRyiUTasgpk7', function=Function(arguments='{"location":"Istanbul","format":"celsius"}', name='get_current_weather'), type='function')]
Fuunction get_current_weather is running...


<b>16°C°F Precipitation: 0%Humidity: 59%Wind: 11 km/hWeatherMonday 15:00Cloudy<b>

----------------------------------------------------------------------------------------------------
the most popular traits among the succesful founders and weather in istanbul
----------------------------------------------------------------------------------------------------
[ChatCompletionMessageToolCall(id='call_axtAtYJEn6tImhpdIfxNdb9h', function=Function(arguments='{"user_input": "most popular traits among successful founders", "index_number": 1}', name='get_llama_response'), type='function'), ChatCompletionMessageToolCall(id='call_CGfjCaY5CG5OdmpNHoY18GVW', function=Function(arguments='{"location": "Istanbul", "format": "celsius"}', name='get_current_weather'), type='function')]
Fuunction get_llama_response is running...


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<b>Based on the study you provided, the most popular traits among successful founders are:

1. Resilience: The ability to bounce back from failures and setbacks is crucial for successful founders.
2. Passion: A deep passion for the startup's idea or problem is essential for maintaining motivation and drive.
3. Network: Building a strong network of contacts and connections can help founders gain access to resources, knowledge, and potential investors.
4. Adaptability: The ability to adapt quickly to changing circumstances and pivot the startup's strategy when necessary is important for success.
5. Vision: A clear and compelling vision for the startup is essential for attracting and motivating team members, investors, and customers.
6. Leadership: Strong leadership skills are necessary for inspiring and guiding a team towards a common goal.
7. Problem-solving: The ability to identify and solve problems quickly and effectively is crucial for overcoming obstacles and achieving success.
8. Creativity: The ability to think creatively and come up with innovative solutions to problems is important for differentiating the startup from competitors.
9.<b>

Fuunction get_current_weather is running...


<b>16°C°F Precipitation: 0%Humidity: 59%Wind: 11 km/hWeatherMonday 15:00Cloudy<b>

<b>16°C°F Precipitation: 0%Humidity: 59%Wind: 11 km/hWeatherMonday 15:00Cloudy<b>

----------------------------------------------------------------------------------------------------
exit
